In [1]:
import gradio as gr
import pandas as pd
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import CSVLoader
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.schema.output_parser import OutputParserException
from langchain_core.documents import Document
from langchain.embeddings import HuggingFaceEmbeddings
from pydantic import BaseModel
import os
from langchain_groq import ChatGroq
from pydantic import BaseModel, HttpUrl, Field
from typing import List
from dotenv import load_dotenv

In [2]:
# Load and preprocess data
loader = CSVLoader(file_path="jiomart_products_database.csv", source_column="title")
documents_raw = loader.load()

In [3]:
documents = []
for doc in documents_raw:
    try:
        row_data = dict(
            line.split(":", 1) for line in doc.page_content.split("\n") if ":" in line
        )
        row_data = {k.strip(): v.strip() for k, v in row_data.items()}

        page_text = f"Name: {row_data.get('title', '')} | Sub-type: {row_data.get('subType', '')} | Type: {row_data.get('type', '')} | Price: {row_data.get('discountedPrice', 0)} | Image: {row_data.get('filename', '')}"
        metadata = {
            "category": row_data.get("type", ""),
            "sub_category": row_data.get("subType", "")
        }

        documents.append(Document(page_content=page_text, metadata=metadata))

    except Exception as e:
        print("Skipping row due to error:", e)

In [4]:
documents

[Document(metadata={'category': 'Staples', 'sub_category': 'Atta, Flours and Sooji'}, page_content='Name: Besan 1 kg | Sub-type: Atta, Flours and Sooji | Type: Staples | Price: 74 | Image: https://www.jiomart.com/images/product/150x150/491349649/besan-1-kg-0-20210521.jpg'),
 Document(metadata={'category': 'Staples', 'sub_category': 'Atta, Flours and Sooji'}, page_content='Name: Besan 500 g | Sub-type: Atta, Flours and Sooji | Type: Staples | Price: 37 | Image: https://www.jiomart.com/images/product/150x150/491349648/besan-500-g-0-20210521.jpg'),
 Document(metadata={'category': 'Staples', 'sub_category': 'Atta, Flours and Sooji'}, page_content='Name: Maida 1 kg | Sub-type: Atta, Flours and Sooji | Type: Staples | Price: 32 | Image: https://www.jiomart.com/images/product/150x150/491349662/maida-1-kg-0-20210521.jpg'),
 Document(metadata={'category': 'Staples', 'sub_category': 'Atta, Flours and Sooji'}, page_content='Name: Rawa 1 kg | Sub-type: Atta, Flours and Sooji | Type: Staples | Pric

In [5]:
len(documents)


5672

In [6]:
load_dotenv( override=True)

True

In [7]:
import getpass
if "OPEN_API_KEY" not in os.environ:
    os.environ["OPEN_API_KEY"] = getpass.getpass("OPEN_API_KEY: ")

In [8]:
if os.environ["OPEN_API_KEY"]:
    print(f"Groq API Key exists and begins {os.environ["OPEN_API_KEY"][:4]}")
else:
    print("Groq API Key not set (and this is optional)")

Groq API Key not set (and this is optional)


In [9]:
from langchain.chat_models import init_chat_model
# model_name = "llama-3.1-8b-instant" ##set Llama 3.1 8B as the LLM for this lab
model_name = "gpt-4o-mini"

In [10]:
from dotenv import load_dotenv
import os

load_dotenv() 
OPEN_API_KEY = os.getenv("OPEN_API_KEY")



In [11]:
#Initialize LLM
llm = init_chat_model(model_name, 
                      model_provider="openai")

convert each product document text into numerical vectors for semantic search.

In [12]:
# embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

from langchain.embeddings import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")


C:\Users\jitesh\AppData\Local\Temp\ipykernel_25396\2867616658.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")


store document embeddings and provide a .retriever() that returns the top-k most relevant documents for a query.

Important details: k controls how many documents are retrieved for context. Typical values: 3–8.


In [13]:
# first_100_docs = documents[:100]
# chroma_store = Chroma.from_documents(documents=first_100_docs, embedding=embedding_model)

chroma_store = Chroma.from_documents(documents=documents, embedding=embedding_model)
retriever = chroma_store.as_retriever()

In [14]:
from langchain.vectorstores import Chroma

# 1) Use a persist directory so the index survives restarts
PERSIST_DIR = "chroma_db"

# 2) Choose a collection name so you can manage multiple corpora
COLLECTION_NAME = "product_docs"

# 3) Create the Chroma vectorstore (documents should be list of LangChain Document objects)
chroma_store = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,          # your OpenAIEmbeddings instance
    persist_directory=PERSIST_DIR,
    collection_name=COLLECTION_NAME,
    # optional: pass client settings if you need more advanced config (see Chroma docs)
)


# 4) Build a retriever with sane defaults
retriever = chroma_store.as_retriever(
    search_type="similarity",                # or "mmr" if you want maximal marginal relevance
    search_kwargs={
        "k": 6,                              # number of docs returned to your RAG pipeline
        # "fetch_k": 12,                     # if supported by your Chroma/LangChain version: fetch more then re-rank
        # "distance_metric": "cosine"       # optional depending on Chroma client version
    }
)


In [15]:
class GroceryItem(BaseModel):
    item_name: str = Field(..., description="Name of the grocery item")
    price: float = Field(..., gt=0, description="Price of the item in INR or USD")
    quantity: int = Field(..., ge=1, description="Quantity of the item")
    image_url: HttpUrl = Field(..., description="Image URL of the grocery item")


class GroceryOutput(BaseModel):
    reasoning: str = Field(..., description="Reasoning for selecting the grocery items")
    items: List[GroceryItem] = Field(..., description="List of selected grocery items")

Purpose: force the LLM to return structured data that you can programmatically use (e.g., to build a shopping cart, display items, parse quantities).

Watchouts:

LLMs can still hallucinate or slightly deviate from strict JSON — always handle OutputParserException and fallback to safe parsing.

The prompt should include explicit examples and a strict output format directive.

In [16]:
parser = PydanticOutputParser(pydantic_object=GroceryOutput)
format_instructions = parser.get_format_instructions()

In [17]:
model_choice = "gpt-4o-mini"

In [18]:
def generate_cart(model_choice, user_input):
    # Step 1: Retrieve context from retriever
    context_docs = retriever.get_relevant_documents(user_input["preferences"])
    relevant_text = "\n".join(doc.page_content for doc in context_docs)

    # Step 2: Format the prompt
    prompt_template = ChatPromptTemplate.from_template("""
    You are a helpful grocery assistant.

    Preferences: {preferences}
    Context: {context}

    Based on the above, suggest a list of grocery items.

    {format_instructions}
    """)

    prompt = prompt_template.format_messages(
        preferences=user_input["preferences"],
        context=relevant_text,
        format_instructions=format_instructions
    )

    # Step 3: Call OpenAI LLM  (NOT Groq)
    llm = init_chat_model(
        model_choice,
        model_provider="openai",
        temperature=0
    )
    
    response = llm.invoke(prompt)

    # Step 4: Parse output with your Pydantic parser
    try:
        result = parser.parse(response.content)
    except OutputParserException:
        return "Could not parse output.", None

    return result


In [19]:
user_input = {
    "preferences": "Vegan, high-protein, gluten-free snacks"
}

result = generate_cart(model_choice, user_input)
type(result)

C:\Users\jitesh\AppData\Local\Temp\ipykernel_25396\508648967.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context_docs = retriever.get_relevant_documents(user_input["preferences"])


__main__.GroceryOutput

In [20]:
# for item in result.items:
#     print(item.item_name)

In [21]:
model_choice = "gpt-4o-mini"   # fixed model you are using

def gradio_interface(preferences, temperature):
    user_input = {
        "preferences": preferences,
        "temperature": temperature,
    }

    # Only pass the fixed model_choice
    result = generate_cart(model_choice, user_input)

    if not result or isinstance(result, str):
        return result, None

    if isinstance(result, tuple):
        explanation, _ = result
        return explanation, None

    # Build a gallery format: [(image_url, caption), ...]
    gallery_items = [
        (str(item.image_url),
         f"{item.item_name}\nQty.{item.quantity}\n₹{item.quantity * item.price}\n")
        for item in result.items
    ]

    return result.reasoning, gallery_items


demo = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(
            label="Describe your grocery needs (e.g., '1. high protein, no besan or curd\n2. Vegan, high-protein, gluten-free snacks')"
        ),
        gr.Slider(
            minimum=0.0, maximum=1.5, value=0.7, step=0.1, label="Temperature"
        )
    ],
    outputs=[
        gr.Textbox(label="Considerations"),
        gr.Gallery(label="Shopping Cart", columns=3, height="auto")
    ],
    title="Smart Grocery Cart Assistant",
    description="Get a product list tailored to your dietary preferences."
)


In [22]:
if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [27]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness, answer_relevancy,
    context_precision, context_recall
)

# 1. Build evaluation dataset by running your pipeline
questions = [
    "high protein snacks.",
    "Vegan, high-protein, gluten-free snacks.",
]

# # ground-truth answers
answers = [
    "RiteBite Max protein Spanish Tomato Chips",
    "RiteBite Max Protein Choco Almond Workout Bar Sugar Free 50 g; RiteBite Max Protein Spanish Tomato Chips 60 g",
]


contexts = []
predicted_answers = []

for q in questions:
    final_output = generate_cart("gpt-4o-mini", {"preferences": q, "temperature": 0.0})

    # Extract model response
    if isinstance(final_output, tuple):
        predicted_answers.append(final_output[0])
    else:
        names = [item.item_name for item in final_output.items]
        predicted_answers.append("; ".join(names))

    # Retrieve context docs
    docs = retriever.get_relevant_documents(q)
    contexts.append([d.page_content for d in docs])

# Build dataset
eval_dataset = Dataset.from_dict({
    "question": questions,
    "reference": answers,              # ground truth
    "contexts": contexts,              # retrieved docs
    "response": predicted_answers      # model output
})
print(eval_dataset)

results = evaluate(
    eval_dataset,
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall],
    llm=llm,
    embeddings=embedding_model
)

# 3. See results
df = results.to_pandas()
print(df)

# df.to_excel("ragas_evaluation_results23.xlsx", index=False)

Dataset({
    features: ['question', 'reference', 'contexts', 'response'],
    num_rows: 2
})


Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


                                 user_input  \
0                      high protein snacks.   
1  Vegan, high-protein, gluten-free snacks.   

                                  retrieved_contexts  \
0  [Name: RiteBite Max Protein Cheese & Jalapeno ...   
1  [Name: Taali Tomato Basil Protein Puffs 60 g |...   

                                            response  \
0  RiteBite Max Protein Cheese & Jalapeno Chips; ...   
1  Taali Tomato Basil Protein Puffs 60 g; Taali C...   

                                           reference  faithfulness  \
0          RiteBite Max protein Spanish Tomato Chips           0.0   
1  RiteBite Max Protein Choco Almond Workout Bar ...           1.0   

   answer_relevancy  context_precision  context_recall  
0          0.408311                0.0             0.0  
1          0.493891                0.0             0.0  


In [28]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness, answer_relevancy,
    context_precision, context_recall
)

# 1. Build evaluation dataset by running your pipeline
questions = [
    "chocolate biscuits",
    "high protein snacks",
]

# ground-truth answers
answers = [
    "cadbuary oreo chocolate cream biscuits;Tasty Treat Chocolate Cream Wafer Biscuits;Cadbuary Bournvita Chocolate Biscuits;Tiffany Sugarfree Chocolate Cream Biscuits",
    "RiteBite Max protein Cheese & Jalapeno Chips; Bite Max Protein Professional Choco Almond Bar Meal Replacement Bar",
]

contexts = []
predicted_answers = []

for q in questions:
    final_output = generate_cart("gpt-4o-mini", {"preferences": q, "temperature": 0.0})

    # Extract model response
    if isinstance(final_output, tuple):
        predicted_answers.append(final_output[0])
    else:
        names = [item.item_name for item in final_output.items]
        predicted_answers.append("; ".join(names))

    # Retrieve context docs
    docs = retriever.get_relevant_documents(q)
    contexts.append([d.page_content for d in docs])

# Build dataset
eval_dataset = Dataset.from_dict({
    "question": questions,
    "reference": answers,              # ground truth
    "contexts": contexts,              # retrieved docs
    "response": predicted_answers      # model output
})

results = evaluate(
    eval_dataset,
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall],
    llm=llm,
    embeddings=embedding_model
)

# 3. See results
df = results.to_pandas()
df

Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,chocolate biscuits,[Name: Tasty Treat Chocolate Cream Wafer Biscu...,Tasty Treat Chocolate Cream Wafer Biscuits 75 ...,cadbuary oreo chocolate cream biscuits;Tasty T...,1.0,0.781630,1.0,0.5
1,high protein snacks,[Name: RiteBite Max Protein Cheese & Jalapeno ...,RiteBite Max Protein Cheese & Jalapeno Chips 6...,RiteBite Max protein Cheese & Jalapeno Chips; ...,1.0,0.399834,1.0,0.5
